In [61]:
STATECODES = {
    'AL':'01',
    'AK':'02',
    'AZ':'04',
    'AR':'05',
    'CA':'06',
    'CO':'08',
    'CT':'09',
    'DE':'10',
    'DC':'11',
    'FL':'12',
    'GA':'13',
    'HI':'15',
    'ID':'16',
    'IL':'17',
    'IN':'18',
    'IA':'19',
    'KS':'20',
    'KY':'21',
    'LA':'22',
    'ME':'23',
    'MD':'24',
    'MA':'25',
    'MI':'26',
    'MN':'27',
    'MS':'28',
    'MO':'29',
    'MT':'30',
    'NE':'31',
    'NV':'32',
    'NH':'33',
    'NJ':'34',
    'NM':'35',
    'NY':'36',
    'NC':'37',
    'ND':'38',
    'OH':'39',
    'OK':'40',
    'OR':'41',
    'PA':'42',
    'RI':'44',
    'SC':'45',
    'SD':'46',
    'TN':'47',
    'TX':'48',
    'UT':'49',
    'VT':'50',
    'VA':'51',
    'WA':'53',
    'WV':'54',
    'WI':'55',
    'WY':'56'
}

In [62]:
USLIST = ['United States', 'U.S.', 'U.S', 'US', 'USA', 'U.S.A', 'America', 'United States of America', 
          'AMERICA', 'UNITED STATES OF', 'United States-', 'United States of America.,10', 'United States.…', 
         'UNITED STATES', 'United states', 'america']
DCLIST = ['Washington D.C.', 'DC', 'D.C.', 'Washington DC', 'DC.', 'D.C.…']
MEXICOLIST = ['Mexico', 'Border', 'Wall', 'Southern Border']
FLLIST = ['Florida', 'FL', 'FL—they’re']
DELAWARELIST = ['Delaware', 'Delaware County']
TXLIST = ['Texas', 'Great States of Texas']
DALLASLIST = ['Dallas', 'Dallas County']
DMLIST = ['Des Moines', 'Des Moines Area']
KYList = ['Kentucky', 'KY']
LALIST = ['Los Angeles', 'LA', 'L.A.']
MCLIST = ['Mason City','Mason City Iowa','Mason City June']
MELIST = ['Middle East', 'Middle-East', 'Mideast']
NYCLIST = ['New York City', 'N.Y.',
             'N.Y. City',
             'N.Y.C.']
NKLIST = ['North Korea', 'N.K.', 'NK']
NHLIST = ['New Hampshire', 'NH', 'N.H.', 'New Hampshire-']
PLACELISTS = [USLIST, DCLIST, MEXICOLIST, FLLIST, DELAWARELIST, TXLIST, DALLASLIST, DMLIST, KYList, LALIST, MCLIST, NYCLIST, 
             MELIST, NKLIST, NHLIST]
def reformat(place):
    for pl in PLACELISTS:
        if place in pl:
            return pl[0]
    return place

In [63]:
import json
from pprint import pprint
from collections import defaultdict 


data = json.load(open('data/trumpTweetPlacesSince2014.json'))

places = []
for row in data:
    for tweet in row:
        if tweet['confidence'] > 0.5:
            places.append(tweet['text'])

places_dict = defaultdict(int)
for p in places:
    p = reformat(p)
    places_dict[p] += 1


with open('tmp/tmp_places_dict.json', 'w+') as f:
    json.dump(places_dict, f)

In [64]:
from geopy.geocoders import Nominatim
from geopy import geocoders
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
geolocator = geocoders.GeoNames(username="kwhitney")

final = defaultdict(int)
for key in places_dict.keys():
    try:
        location = geolocator.geocode(key, timeout=None)
        #location = gn.geocode(key, timeout=None)
        if location:
            loc = location.raw
            cId = loc.get('countryId')
            if cId and cId == '6252001':
                state_name = loc['adminCode1']
                if state_name in STATECODES.keys():
                    final[state_name] += places_dict[key]
    except GeocoderTimedOut as e:
        print("failed for " + key)
        continue

failed for Mexico
failed for Canada
failed for Northern Border
failed for Harrisburg


In [65]:
# with open('staterules.json','r') as f:
#     rls = json.load(f)
    
# newcodes = {v: k for k, v in STATECODES.items()}

# new_freq_dict = {}

# for k,v in rls.items():
#     new_freq_dict[newcodes[k]] = v

# print(new_freq_dict)


In [68]:
import csv
with open('data/state_to_freq.csv', 'w+') as csvfile:
    wrtr = csv.writer(csvfile)
    wrtr.writerow(['state', 'freq'])
    for k,v in final.items():
        wrtr.writerow([k,v])

In [69]:
# Import libraries
import pandas as pd
import folium
import os

# Load the shape of the zone (US states)
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data
# You have to download this file and set the directory where you saved it
state_geo = 'data/us-states.json'
 
# Load the unemployment value of each state
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data
state_unemployment = 'data/state_to_freq.csv'
state_data = pd.read_csv(state_unemployment)
 
# Initialize the map:
m = folium.Map(location=[37, -102], zoom_start=5)
 
# Add the color for the chloropleth:
m.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=state_data,
 columns=['state', 'freq'],
 key_on='feature.id',
 fill_color='BuPu',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Times Mentioned'
)
folium.LayerControl().add_to(m)
 
# Save to html
m.save('output/StatesMentioned2.html')
